In [132]:
import pandas as pd
import datetime as dt
import sys
import hashlib as hs
import base64 as b64
import numpy as np
import os
import darc_core
from darc_core.metrics import Metrics 
from darc_core.utils import M_COL, T_COL, T_COL_IT, NB_GUESS, SIZE_POOL
from darc_core.preprocessing import round1_preprocessing
from darc_core.utils import check_format_trans_file

In [133]:
df = pd.read_csv("../Master/data/ground_truth.csv", parse_dates=["date"])
df.id_user=df.id_user.astype(str)
df_copy=df.copy()
df_copy.head()

,id_user,date,hours,id_item,price,qty
0,17850,2010-12-01,08:26,85123A,2.55,6
1,17850,2010-12-01,08:26,71053,3.39,6
2,17850,2010-12-01,08:26,84406B,2.75,8
3,17850,2010-12-01,08:26,84029G,3.39,6
4,17850,2010-12-01,08:26,84029E,3.39,6


In [115]:
def generateSalts():
    # Set of {1..12} (number of the month)
    months = pd.DatetimeIndex(df_copy["date"]).month.unique()
    ordered_months = set(reversed(months))

    # Set of all unique user's IDs
    ids = df_copy["id_user"].unique()
    # Transform YYYY-MM-DD to MM
    df_copy["date"] = pd.DatetimeIndex(df_copy["date"]).month

    salt_table=pd.DataFrame(columns=ordered_months,index=ids)
    salt_table.set_index(ids,inplace=True)

    # Generate salt table
    for i in ids:
        for j in ordered_months:
            # Generates unique b64 values
            salt_table[j][i]=os.urandom(256)
    
    return salt_table

salt_table = generateSalts()
salt_table.head()

,1,2,3,4,5,6,7,8,9,10,11,12
17850,b'\x92\xf1_|d}5\xac\xf5\x1f\xc6\x1ch-N\xfdlV\x...,"b""\xd1\xdc:\xe3C\xf1\xd0|}\xd0\xae\x06\x9c\x90...","b'EGJ8\x11PeTU\xec""\xfb\xba\x1f\x10F\xfd\xb5m\...",b'Ov\n\xbeA\x14y\rt\xafXu\x9a\xa7+\x05\x95:J\x...,b'#\x9b\\\x19\xcf]0\x81\x82yN\xf2d.3F&\xea\xbc...,b'\xbf^u\xf9\xf3\x1a\x92J\x9aw\xf9\xee[G\xf0nr...,"b""8~\x18\xc0\xeaB\x80%@a\x08\xd7\x9a\x91\xa8\x...",b'\xf9ul\xb9B\x1ao\xd7\xe13\x8a\xf6\xc5\xad\xe...,"b""b&\xa71J]D\xe35\x9d\xf7\xa7%\xb3\xb7\x90\xd6...",b'\xd8\xd4\x88\xfaXJb\x1eBA\x01w\xb4\xd9\xce.4...,b'\xafc\xc9k{y\xfd\x9a4/-\xe8h\xe6\x87\x1f$\xc...,b'\xf0T\x95\xb9gd\x95\xe0\xc7\xc3\x9cP\xfa\x0c...
13047,"b""z@\x9c\xd1q}\x18\x931&\xf7\xf0n\xad-pE5\xb5\...","b'\xfb]\'\x1f\xe1.""\xc1U\xe7\x1e\xc6\xc1#7\xf6...",b'uU\x9cM_2\x880A\x81\xba\x07\xbc\xe0\xcb\x8dq...,b'\xcap\xddo\xc9\xe8&#\xd7:\x13\xf4J\x1d\xc1\x...,b'l\xf3\xad\x12\xfc\xf3\xab\xeb(\xb45]\xff[ph\...,"b'\xf42""\xc2\xfd%\xe5\xa1\xa0\xb9\x17\xf1\x82|...","b""L\xc4\xee0\xecI\xd618\xdd\xb1\x05\xcb\xd0\xa...","b"":J\x81\x0c<E\tNM\x8a:Kj\x8d\x0f\x02E\xff\xd3...",b'\x04G\xdd0\xd8G7\xa3Y1T\xf3\x15\xae\x17s \xf...,b'\xc2\x80\x81>\x89\x90#S\xdf|\xd7\x14)\x1d\x0...,b'Q\xfa\xb3\xb9\xc8\xcb\x9c4\x9dB>3A\xd4c\x96\...,b'\xce;\xe5\x08\x85y\x16\x94;\x1cQ\xf3\xb8\xeb...
12583,b'C^\xb6\xb6A\x15\x93\t*\xc8?\xe9\xe7\xe1W\x03...,b' \xa6\x84\xfd\xf4\xc2\xe1+<\xea\xe0\xed\xc49...,b'\x94]\xea\x98\x16{\x1f\x98\x99\x85K\xf5\xee\...,"b""u\xf6\x96\xf5g.P\xc8U(V\xfb_22\x91\xa0y\xb3\...",b'%\xce\xf1\r%\n[P\x97\xe2B]\xdb\xf2\x11\xff\x...,b'\xfaS\xe1\xd4\xed\x8e\xcc\xe1<\xb6X\xc7\x89q...,b'\x90R\xff\xfc\x94\xb5O[\x85\x0c\xb11\x86p\xc...,b'\x1bc\xb9\xbb\x85f\x04\xea\xa0$\xb6\x85\xa21...,"b""\xf5k\xed\x95\xee\xcf\xae\x0f\x18.\xa8\xbbO\...",b'\xabE\xd1O\xdf\xb4S\x9aI\x1aL\xe7\xa5\xb5h\x...,b'\xf9|\xe9C.=\xc2?O\xf8\xc6\x8fy\xe2\x81\xe8]...,b'\xefKQB\\M\x00\xda\x00\xe8&X\x88\xcf8\xec\xd...
13748,"b'\xc4\x10\xfd\x02\xbaC6\xbe""-\x01\x02\xca\xd5...",b'\x8en\x19\xce&\xd9cr\xac5\xe1\x84z\xf1\xf8\x...,"b'\xb5\x17\xcb\xce>\xf3\x14""?@\xe0g\tH3\xdeF\x...","b""\xffP\x04\xbd\xff\x14\x1dc\xef\xfd\x90\xc7]\...",b'\xa8\xd6\xc2Y\x01]\xa4P\xfc/g8\xd3d\xeb\xb1\...,b'^;\xaf\xff\xf8^\xb9\xb3\xf8\xb9\r\x037\x97\x...,b'\xf6\xc9m.\xb0\xeb\x1ao\t\x7fr3\x15\x11\xb3\...,b'rB9B\x9f\x90\xcc\xe8\xc3\xbf\xdb2\xb4\x87n\x...,"b""/>\x81\xb7\x0e>\xbb\xfc\xa7\xa7\xdb\xa7\xd4\...",b'}1\x1c\xb8~\xa1\xc3%\x18q\x7f\xcc\xfa\xe4\x8...,b'\xed\x16\x04\x01H+\xeeT6\xc6\x0c\xc6?$\xd0\x...,b'\x0e\x9a\x16\xb7\x17\x92\x0fOC\xdf\xfa\x87P\...
15100,"b""M\xba\x17\x91\xcd<o\x04n\xa2m\xdc\xe7\xcf\xf...","b""\xc9\xd2\xc3\xf0\xd4\xe6\x9bwmn\x93\xbe3\xa3...",b'Y\xdb\xb8\xeb*\xa6\x908\xe8o\xc6\xe6\x08y\xa...,"b'\xd6\x8f,\x12\xcb\xd1\xbe\xad\xe2\x05""\xca\x...",b'\x95J5\xa2wg\xd9PW\xae\xc5\x9b\xcb\xf5+\x1a\...,b'B\xa30\x1a\x03\xd8Q\x03\x07\xe8\xa7\x02\xaa\...,"b'\xbeM\xf3""\x10\x81\xa5\x8d\xddj\xb1\xc3\xce(...",b'S\x97\x99\xe3\n\xc98\x9fV\xb6@\x92q\x92~S\xc...,"b""\xbd\x82'r\x0f\x0c\x99b]a\x82;\xe1zZ\xe5\xc1...","b""\xdc\x9f\x8b\x85\xa7n=Iq\xfa\xba\x07\x8bx\x1...",b't>8\x14\xd1\xdc?\xb6\xaae:\xa20OJ\xda/+\x94\...,b'\xb9\x84?j\xb3c\x07\x00\xe1N\xd7Z\x19\xd5\xb...


In [134]:
def hashme(id_user, month): 
    hashme.counter+=1
    percent=(hashme.counter/307054)*100 
    sys.stdout.write("\rProgress %i -- Count : %i / 307054" % (percent,hashme.counter))
    sys.stdout.flush()
    return hs.sha256(salt_table.loc[[id_user],month][0] + id_user.encode()).hexdigest()
#Static Counter for ProgressBar
hashme.counter=0

In [135]:
def doHash():
    df_copy["month"] = df["date"].transform(lambda x: x.month) 
    df_copy.sort_values(["id_user","month"])
    salt_table.sort_index(inplace=True)
    df["id_user"]=df_copy.apply(lambda x: hashme(x["id_user"],x["month"]), axis=1)

In [136]:
%%time
doHash()

Progress 100 -- Count : 307054 / 307054CPU times: user 15min 24s, sys: 2min 13s, total: 17min 37s
Wall time: 33min 39s


In [138]:
df["year"] = df["date"].transform(lambda x: x.year) 
df["month"] = df["date"].transform(lambda x: x.month) 
df["day"] = df["date"].transform(lambda x: x.day) 

In [4]:
def transformDate(year,month,day):
    if (day==31 or day==30):
        day=29
    if((day%10)<5):
        inf = (int)(day/10)*10
        sup = (int)(day/10)*10 + 5
    else:
        inf = (int)(day/10)*10 + 5
        sup = (1+(int)(day/10))*10  
    return dt.date(year,month,(int)((inf+sup)/2))

df["new_date"]=df.apply(lambda x: transformDate(x["year"],x["month"],x["day"]), axis=1)
df["new_date"].value_counts()

KeyboardInterrupt: 

In [139]:
def transformDate(year,month,day):
    if((day%2)==0):
        day-=1   
    return dt.date(year,month,day)
df["date"]=df.apply(lambda x: transformDate(x["year"],x["month"],x["day"]), axis=1)
df

,id_user,date,hours,id_item,price,qty,year,month,day
0,20ab495b4944e61088068e9a1587cffc73ed506e93fead...,2010-12-01,08:26,85123A,2.55,6,2010,12,1
1,20ab495b4944e61088068e9a1587cffc73ed506e93fead...,2010-12-01,08:26,71053,3.39,6,2010,12,1
2,20ab495b4944e61088068e9a1587cffc73ed506e93fead...,2010-12-01,08:26,84406B,2.75,8,2010,12,1
3,20ab495b4944e61088068e9a1587cffc73ed506e93fead...,2010-12-01,08:26,84029G,3.39,6,2010,12,1
4,20ab495b4944e61088068e9a1587cffc73ed506e93fead...,2010-12-01,08:26,84029E,3.39,6,2010,12,1
5,20ab495b4944e61088068e9a1587cffc73ed506e93fead...,2010-12-01,08:26,22752,7.65,2,2010,12,1
6,20ab495b4944e61088068e9a1587cffc73ed506e93fead...,2010-12-01,08:26,21730,4.25,6,2010,12,1
7,20ab495b4944e61088068e9a1587cffc73ed506e93fead...,2010-12-01,08:28,22633,1.85,6,2010,12,1
8,20ab495b4944e61088068e9a1587cffc73ed506e93fead...,2010-12-01,08:28,22632,1.85,6,2010,12,1
9,9682bb6424edc103cab4c009e25fb4ca7b4b271a1a6635...,2010-12-01,08:34,84879,1.69,32,2010,12,1


In [143]:
df.drop(["year","month","day"],axis=1, inplace=True)
df["date"]=df["date"].apply(lambda x: x.strftime('%Y/%m/%d'))
df.to_csv("./oto.csv",index=False)

In [140]:
# Anonymize hours by Truncate
df["hours"]= pd.to_datetime(df["hours"], format = '%H:%M%S')
df["hours"]=df["hours"].dt.hour

In [145]:
# Group qty and price by Mean()
dates = df["date"].unique()
hours=df["hours"].unique()
for d in dates:
    for h in hours:
        mask=(df["date"]==d) & (df["hours"]==h)
        df.loc[mask,"qty"]=df[mask]["qty"].mean()
        df.loc[mask,"price"]=df[mask]["price"].mean()

In [147]:
df.head()
df.to_csv("./at1.csv",index=False)

In [7]:
ground = pd.read_csv("./ground_truth.csv")
ground.head(3)

,id_user,date,hours,id_item,price,qty
0,17850,2010/12/01,08:26,85123A,2.55,6
1,17850,2010/12/01,08:26,71053,3.39,6
2,17850,2010/12/01,08:26,84406B,2.75,8


In [163]:
da = pd.read_csv("./at1.csv")
da.head(3)

,id_user,date,hours,id_item,price,qty
0,20ab495b4944e61088068e9a1587cffc73ed506e93fead...,2010/12/01,8,85123A,2.55,9.867347
1,20ab495b4944e61088068e9a1587cffc73ed506e93fead...,2010/12/01,8,71053,3.39,9.867347
2,20ab495b4944e61088068e9a1587cffc73ed506e93fead...,2010/12/01,8,84406B,2.75,9.867347


In [226]:
ground_truth, submission = round1_preprocessing(
    "./ground_truth.csv", "./at1.csv"
)

# Check the format of the Anonymized Transaction file
check_format_trans_file(ground_truth, submission)

In [227]:
%%time
metric = Metrics(ground_truth, submission)
#scores = metric.scores()
#scores

CPU times: user 1min 33s, sys: 1.73 s, total: 1min 34s
Wall time: 1min 50s


In [167]:
scores

[0.9544850393418423,
 0,
 0.7867699526022446,
 0.0160489791,
 0.0,
 0.0,
 0.0,
 0.03264,
 0.0,
 0.259542,
 0.0,
 0.301483,
 0.970402]

In [223]:
date_col = metric._gt_t_col_it['date']
qty_col = metric._gt_t_col_it['qty']

f_hat = metric._evaluate([date_col, qty_col])

In [225]:
# Show reidentified lines
for h in range(0,4034):
    array = f_hat.iloc[h]
    for x in range(0,13):
        if array[x] != "DEL":
            print(f_hat.iloc[h])

id_user    12347
0          13963
1          15661
2            DEL
3            DEL
4          16255
5            DEL
6          18041
7            DEL
8          13850
9            DEL
10         18055
11           DEL
12         13534
Name: 0, dtype: object
id_user    12347
0          13963
1          15661
2            DEL
3            DEL
4          16255
5            DEL
6          18041
7            DEL
8          13850
9            DEL
10         18055
11           DEL
12         13534
Name: 0, dtype: object
id_user    12347
0          13963
1          15661
2            DEL
3            DEL
4          16255
5            DEL
6          18041
7            DEL
8          13850
9            DEL
10         18055
11           DEL
12         13534
Name: 0, dtype: object
id_user    12347
0          13963
1          15661
2            DEL
3            DEL
4          16255
5            DEL
6          18041
7            DEL
8          13850
9            DEL
10         18055
11           

Name: 58, dtype: object
id_user    12423
0          16270
1            DEL
2          16980
3          16791
4          16469
5          14817
6          15916
7            DEL
8          16780
9            DEL
10           DEL
11           DEL
12         14446
Name: 58, dtype: object
id_user    12423
0          16270
1            DEL
2          16980
3          16791
4          16469
5          14817
6          15916
7            DEL
8          16780
9            DEL
10           DEL
11           DEL
12         14446
Name: 58, dtype: object
id_user    12423
0          16270
1            DEL
2          16980
3          16791
4          16469
5          14817
6          15916
7            DEL
8          16780
9            DEL
10           DEL
11           DEL
12         14446
Name: 58, dtype: object
id_user    12423
0          16270
1            DEL
2          16980
3          16791
4          16469
5          14817
6          15916
7            DEL
8          16780
9            DEL
10 

Name: 101, dtype: object
id_user    12477
0            DEL
1            DEL
2          17512
3            DEL
4          13772
5            DEL
6            DEL
7            DEL
8          12477
9            DEL
10         15478
11         12477
12           DEL
Name: 101, dtype: object
id_user    12477
0            DEL
1            DEL
2          17512
3            DEL
4          13772
5            DEL
6            DEL
7            DEL
8          12477
9            DEL
10         15478
11         12477
12           DEL
Name: 101, dtype: object
id_user    12477
0            DEL
1            DEL
2          17512
3            DEL
4          13772
5            DEL
6            DEL
7            DEL
8          12477
9            DEL
10         15478
11         12477
12           DEL
Name: 101, dtype: object
id_user    12477
0            DEL
1            DEL
2          17512
3            DEL
4          13772
5            DEL
6            DEL
7            DEL
8          12477
9            DEL

id_user    12539
0            DEL
1          14709
2            DEL
3          14530
4            DEL
5            DEL
6            DEL
7            DEL
8            DEL
9            DEL
10           DEL
11         14034
12           DEL
Name: 149, dtype: object
id_user    12540
0          17293
1          15518
2          16110
3          12540
4            DEL
5          12540
6          14312
7          13814
8          16340
9          15763
10         15140
11         17758
12           DEL
Name: 150, dtype: object
id_user    12540
0          17293
1          15518
2          16110
3          12540
4            DEL
5          12540
6          14312
7          13814
8          16340
9          15763
10         15140
11         17758
12           DEL
Name: 150, dtype: object
id_user    12540
0          17293
1          15518
2          16110
3          12540
4            DEL
5          12540
6          14312
7          13814
8          16340
9          15763
10         15140
11     

id_user    12583
0          15525
1          17284
2          17609
3          17164
4            DEL
5          13831
6          17960
7          14961
8          12841
9          14422
10         17314
11         17450
12         14904
Name: 184, dtype: object
id_user    12583
0          15525
1          17284
2          17609
3          17164
4            DEL
5          13831
6          17960
7          14961
8          12841
9          14422
10         17314
11         17450
12         14904
Name: 184, dtype: object
id_user    12583
0          15525
1          17284
2          17609
3          17164
4            DEL
5          13831
6          17960
7          14961
8          12841
9          14422
10         17314
11         17450
12         14904
Name: 184, dtype: object
id_user    12583
0          15525
1          17284
2          17609
3          17164
4            DEL
5          13831
6          17960
7          14961
8          12841
9          14422
10         17314
11     

...

id_user    13495
0          15290
1            DEL
2            DEL
3            DEL
4          14735
5            DEL
6            DEL
7            DEL
8            DEL
9            DEL
10           DEL
11         14913
12         18055
Name: 796, dtype: object
id_user    13495
0          15290
1            DEL
2            DEL
3            DEL
4          14735
5            DEL
6            DEL
7            DEL
8            DEL
9            DEL
10           DEL
11         14913
12         18055
Name: 796, dtype: object
id_user    13495
0          15290
1            DEL
2            DEL
3            DEL
4          14735
5            DEL
6            DEL
7            DEL
8            DEL
9            DEL
10           DEL
11         14913
12         18055
Name: 796, dtype: object
id_user    13495
0          15290
1            DEL
2            DEL
3            DEL
4          14735
5            DEL
6            DEL
7            DEL
8            DEL
9            DEL
10           DEL
11     

Name: 829, dtype: object
id_user    13534
0          13534
1          17180
2          14475
3          14461
4          14231
5          14191
6          15379
7          12981
8          16340
9            DEL
10         16542
11         15785
12         12826
Name: 829, dtype: object
id_user    13536
0            DEL
1            DEL
2            DEL
3            DEL
4            DEL
5            DEL
6            DEL
7            DEL
8            DEL
9          15429
10         15140
11         15394
12         17644
Name: 830, dtype: object
id_user    13536
0            DEL
1            DEL
2            DEL
3            DEL
4            DEL
5            DEL
6            DEL
7            DEL
8            DEL
9          15429
10         15140
11         15394
12         17644
Name: 830, dtype: object
id_user    13536
0            DEL
1            DEL
2            DEL
3            DEL
4            DEL
5            DEL
6            DEL
7            DEL
8            DEL
9          15429

Name: 862, dtype: object
id_user    13588
0            DEL
1            DEL
2            DEL
3          16712
4            DEL
5            DEL
6            DEL
7          16133
8          14639
9          13321
10         14329
11         14277
12           DEL
Name: 862, dtype: object
id_user    13588
0            DEL
1            DEL
2            DEL
3          16712
4            DEL
5            DEL
6            DEL
7          16133
8          14639
9          13321
10         14329
11         14277
12           DEL
Name: 862, dtype: object
id_user    13588
0            DEL
1            DEL
2            DEL
3          16712
4            DEL
5            DEL
6            DEL
7          16133
8          14639
9          13321
10         14329
11         14277
12           DEL
Name: 862, dtype: object
id_user    13588
0            DEL
1            DEL
2            DEL
3          16712
4            DEL
5            DEL
6            DEL
7          16133
8          14639
9          13321

id_user    13630
0            DEL
1            DEL
2          13630
3            DEL
4          12647
5            DEL
6          13750
7            DEL
8          17183
9          15150
10           DEL
11           DEL
12         13435
Name: 889, dtype: object
id_user    13630
0            DEL
1            DEL
2          13630
3            DEL
4          12647
5            DEL
6          13750
7            DEL
8          17183
9          15150
10           DEL
11           DEL
12         13435
Name: 889, dtype: object
id_user    13630
0            DEL
1            DEL
2          13630
3            DEL
4          12647
5            DEL
6          13750
7            DEL
8          17183
9          15150
10           DEL
11           DEL
12         13435
Name: 889, dtype: object
id_user    13630
0            DEL
1            DEL
2          13630
3            DEL
4          12647
5            DEL
6          13750
7            DEL
8          17183
9          15150
10           DEL
11     

KeyboardInterrupt: 

In [224]:
f_hat

,id_user,0,1,2,3,4,5,6,7,8,9,10,11,12
0,12347,13963,15661,DEL,DEL,16255,DEL,18041,DEL,13850,DEL,18055,DEL,13534
1,12348,15620,17515,DEL,DEL,14546,DEL,DEL,DEL,DEL,16594,DEL,DEL,DEL
2,12349,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,16712,DEL
3,12350,DEL,DEL,14798,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
4,12352,DEL,DEL,15059,12352,DEL,DEL,DEL,DEL,DEL,17783,DEL,14098,DEL
5,12354,DEL,DEL,DEL,DEL,14231,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL
6,12355,DEL,DEL,DEL,DEL,DEL,16672,DEL,DEL,DEL,DEL,DEL,DEL,DEL
7,12356,DEL,15061,DEL,DEL,12356,DEL,DEL,DEL,DEL,DEL,DEL,12449,DEL
8,12357,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,DEL,14511,DEL
9,12358,DEL,DEL,DEL,DEL,DEL,DEL,DEL,15051,DEL,DEL,DEL,DEL,13521


In [162]:
df["price"]=df_copy["price"]
df.to_csv("./at3.csv",index=False)

In [ ]:
Quasi_Ident. Date + Hours ( +/- qty,price)
at1 = without generalized
at2 = qty price generalized
at3 = only qty

In [ ]:
k-anon => classes d'equivalence ^
l-diversity ? & why
=> reidentification
(l-diversity / t-closeness ) <= [ !!  reidentification !! ]
Si reidentification faible alors supprimer des lignes => remplacer